In [1]:
from utils import get_webpage_content, extract_params, process_model_output, filter_json
from json5 import loads
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm

In [ ]:
example_single = "I'm searching for beers with ABV greater than 6%, IBU less than 80, EBC greater than 15, brewed after 01-2025, using yeast named Brettanomyces, hops named Nugget, malt named Chocolate, that go well with fish_and_chips, and named Punk_IPA."
processed_text = extract_params(example_single)
# print('*' + processed_text)
processed_text = process_model_output(processed_text)
# print('**' + processed_text)
processed_text = loads(processed_text)
print(str(processed_text))
print('-----------------')
processed_text_filtered, errors = filter_json(processed_text)
print(str(processed_text_filtered), errors)

In [2]:
with open('test_cases.txt', 'r') as f:
    examples = f.read().split('\n')

examples_processed = [example.split("\"")[1].split("\"")[0] for example in examples]
examples_cases = [example.split("(")[1].split(")")[0] if len(example.split("(")) > 1 else None for example in examples]

In [3]:
examples_list = []
case_list = []
output_list = []
output_filtered_list = []
error_list = []

for i in tqdm(range(5)):
    examples_list.append(examples_processed[i])
    case_list.append(examples_cases[i])
    count = 0
    while count < 2:
        try:
            processed_text = loads(process_model_output(extract_params(examples_processed[i])))
            processed_text_filtered, errors = filter_json(processed_text)
            output_list.append(processed_text)
            output_filtered_list.append(processed_text_filtered)
            error_list.append(errors)
            count = 2
        except:
            count += 1
            if count == 2:
                output_list.append("{}")
                output_filtered_list.append("{}")
                error_list.append("")
            continue

100%|██████████| 5/5 [01:10<00:00, 14.14s/it]


In [4]:
pd.DataFrame({"Example": examples_list, "Edge_Case": case_list, "Output": output_list, "Filtered_Output": output_filtered_list, "Errors": error_list})

,Example,Edge_Case,Output,Filtered_Output,Errors
0,Find a beer with ABV greater than 8%.,None,{'abv_gt': 8},{'abv_gt': 8},
1,I want a beer brewed before 10-2010.,None,{'brewed_before': '10-2010'},{'brewed_before': '10-2010'},
2,I'm looking for beers with ABV less than 5% and brewed after 01-2015.,None,"{'abv_lt': 5, 'brewed_after': '01-2015'}","{'abv_lt': 5, 'brewed_after': '01-2015'}",
3,Give me a beer that contains the yeast Saccharomyces Pastorianus.,None,{'yeast': 'Saccharomyces Pastorianus'},{},
4,I want a beer with IBU greater than 40 and malt Maris Otter Extra Pale.,None,"{'ibu_gt': 40, 'malt': 'Maris Otter Extra Pale'}",{'ibu_gt': 40},


In [31]:
query_string = "&".join([f"{key}={value}" for key, value in output_filtered_list[0].items()] + [f"per_page={80}"])

In [34]:
"https://api.punkapi.com/v2/beers?" + query_string + "&page=1"

'https://api.punkapi.com/v2/beers?abv_gt=8&per_page=80&page=1'

In [20]:
content = get_webpage_content("https://api.punkapi.com/v2/beers?" + query_string)
content = loads(content)

In [21]:
len(content)

80

In [17]:
content

[{'id': 5,
  'name': 'Avery Brown Dredge',
  'tagline': "Bloggers' Imperial Pilsner.",
  'first_brewed': '02/2011',
  'description': 'An Imperial Pilsner in collaboration with beer writers. Tradition. Homage. Revolution. We wanted to showcase the awesome backbone of the Czech brewing tradition, the noble Saaz hop, and also tip our hats to the modern beers that rock our world, and the people who make them.',
  'image_url': 'https://images.punkapi.com/v2/5.png',
  'abv': 7.2,
  'ibu': 59,
  'target_fg': 1027,
  'target_og': 1069,
  'ebc': 10,
  'srm': 5,
  'ph': 4.4,
  'attenuation_level': 67,
  'volume': {'value': 20, 'unit': 'litres'},
  'boil_volume': {'value': 25, 'unit': 'litres'},
  'method': {'mash_temp': [{'temp': {'value': 66, 'unit': 'celsius'},
     'duration': 70}],
   'fermentation': {'temp': {'value': 10, 'unit': 'celsius'}},
   'twist': None},
  'ingredients': {'malt': [{'name': 'Lager Malt',
     'amount': {'value': 6.63, 'unit': 'kilograms'}},
    {'name': 'Wheat', 'amou